## Spectrogram of sleep deprivation vs nsd session 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from plotUtil import Fig
import subjects

In [8]:
%matplotlib widget
figure = Fig()
fig,gs = figure.draw(num=1,grid=(3,1))
sessions = subjects.Sd().ratSday3 + subjects.Nsd().ratSday2
for sub, sess in enumerate(sessions):
    # maze = sess.epochs.maze
    tag = sess.recinfo.animal.tag
    post = sess.epochs.post

    
    ax = plt.subplot(gs[sub])
    sess.viewdata.specgram(chan=18,period=post,ax=ax)
    figure.panel_label(ax,'a')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=2.0, description='Clim :', max=30.0, min=0.1), Dropdown(description='C…

interactive(children=(FloatSlider(value=2.0, description='Clim :', max=30.0, min=0.1), Dropdown(description='C…